# 🎤 Week2: GPT-4o Mini TTS 강의 노트 (Colab)

> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY` 환경 변수를 다시 설정해야 합니다.

## 진행 순서
1. Quickstart: 환경 준비 및 Audio API 소개
2. TTS 모델 라인업과 주요 제약사항
3. 기본 텍스트→음성 변환 데모 (파일 저장)
4. 음성 옵션(voice) · 프롬프트 제어 · 다국어 지원
5. 스트리밍 재생 및 실시간 활용
6. 출력 포맷 비교와 응용 예시
7. 활용 정책·고지 의무 · 실습 과제



## 1) Quickstart: 환경 준비 및 Audio API 소개
Text-to-Speech는 Audio API의 `speech` 엔드포인트를 사용합니다. 모델 ID, 입력 텍스트, 목소리(voice)가 필수 입력이며, 추가적으로 말투를 제어하는 `instructions` 등을 전달할 수 있습니다.


In [ ]:
%pip install openai


In [ ]:
import os
from getpass import getpass

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")


In [ ]:
from openai import OpenAI, AsyncOpenAI
from pathlib import Path

client = OpenAI()
async_client = AsyncOpenAI()
print("✅ 클라이언트 생성 완료")


## 2) TTS 모델 라인업과 주요 제약사항
- **권장 모델**: `gpt-4o-mini-tts` (품질과 지연 균형이 좋고, 감정/억양 제어가 가능)
- **대체 모델**: `tts-1` (낮은 지연, 상대적으로 낮은 품질), `tts-1-hd` (높은 품질, 더 긴 지연)
- **정책 안내**: 생성된 음성이 AI라는 사실을 사용자에게 명시해야 하며, 커스텀 음성/실제 목소리 복제는 지원하지 않습니다.


## 3) 기본 텍스트→음성 변환 데모 (파일 저장)
가장 기본적인 워크플로우는 텍스트와 음성 설정을 넘겨 MP3 파일로 저장하는 것입니다.


In [ ]:
speech_file = Path("speech.mp3")
speech_file.unlink(missing_ok=True)

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build something people love!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file)

print("🎧 음성 파일 저장 완료 ->", speech_file)


> **설명 포인트**
> - `with_streaming_response.create` 컨텍스트를 사용하면 서버로부터 오는 오디오 스트림을 실시간으로 파일에 쓸 수 있습니다.
> - 음성 합성 요청마다 `voice`, `instructions`를 통해 억양/감정/속도 등을 제어할 수 있습니다.



## 4) 음성 옵션(voice) · 프롬프트 제어 · 다국어 지원
- **지원 음성(11종)**: `alloy`, `ash`, `ballad`, `coral`, `echo`, `fable`, `nova`, `onyx`, `sage`, `shimmer` 등 (최신 목록은 OpenAI.fm에서 확인)
- **프롬프트 제어**: 인사톤, 속도, 감정, 억양, 속삭임 등을 `instructions`에 명시
- **언어 지원**: Whisper 모델 수준의 다국어 텍스트 입력을 지원 (음성은 영어에 최적화되어 있으나 대부분의 언어 입력 가능)
- **정책 안내**: 음성이 AI 생성임을 사용자에게 반드시 고지해야 합니다.


## 5) 스트리밍 재생 및 실시간 활용
TTS 응답을 바로 재생하려면 스트리밍 응답과 로컬 플레이어를 결합할 수 있습니다.


In [ ]:
import asyncio
from openai.helpers import LocalAudioPlayer

async def play_streaming_tts(text: str):
    async with async_client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text,
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

# asyncio.run(play_streaming_tts("Today is a wonderful day to build something people love!"))


> **설명 포인트**
> - `response_format="wav"` 또는 `"pcm"`을 사용하면 지연을 줄이고 빠르게 스트리밍 재생할 수 있습니다.
> - 웹/모바일 앱에서는 스트리밍 응답을 스피커로 직접 송출하거나 음성 채팅에 활용할 수 있습니다.



## 6) 출력 포맷 비교와 응용 예시
| 포맷 | 용도 | 특징 |
| --- | --- | --- |
| `mp3` (기본) | 일반적인 배포/저장 | 범용성 높음, 용량 작음 |
| `opus` | 인터넷 스트리밍 | 낮은 지연, 적은 비트레이트 |
| `aac` | 모바일/미디어 플랫폼 | YouTube·iOS·Android 선호 |
| `flac` | 보존·아카이빙 | 무손실 압축, 용량 큼 |
| `wav` | 저지연 재생 | 헤더 있는 비압축, 디코딩 필요 없음 |
| `pcm` | 실시간 파이프라인 | 헤더 없는 원시 샘플(24kHz/16bit) |

> **응용 아이디어**: 프롬프트 자동화로 블로그 TTS, 다국어 오디오 가이드, 실시간 음성 피드백 시스템 등을 구현할 수 있습니다.


## 7) 활용 정책·고지 의무 · 실습 과제
- **정책**: 모든 사용자에게 음성이 AI 생성임을 명확히 고지해야 합니다.
- **커스텀 음성**: 현재는 지원되지 않으며, 실존 인물 음성 복제는 금지됩니다.
- **실습 제안**:
  1. 동일한 텍스트에 대해 여러 음성을 비교하고 특징을 정리
  2. 다국어 입력을 실험해 발음 품질을 관찰
  3. 스트리밍 재생을 구현해 실시간 피드백 앱 UI 목업 제작
- **추가 학습**: [Audio API 문서](https://platform.openai.com/docs/api-reference/audio/createSpeech), [OpenAI.fm](https://openai.fm) 데모, [Whisper 언어 목록](https://github.com/openai/whisper#available-models-and-languages) 참고


> 💡 **강의 마무리**
> - Week2 실습 노트(`03-TTS-Practice.ipynb` 예정)에서 TODO 형태로 동일한 단계(파일 저장, 스트리밍)를 구현하도록 안내하세요.
> - 음성 생성은 빠르므로 라이브 데모가 가능하지만, 다양한 음성/언어 결과를 비교하는 샘플을 미리 준비하면 Discussion에 도움이 됩니다.

